# kerasに慣れよう1
入力ベクトル$(x_{1},x_{2},x_{3},x_{4},x_{5})$ ($x_i \in [0,1]$)の各要素の加算結果が2.5以上で1,未満で0を出すモデル  
$f(x) = if\ \sum^5_{i=1}x_i \ \geq\ 2.5 \ then\  1\  else\  0$  
参考:[無から始めるKeras 第1回][1]
[1]:https://qiita.com/Ishotihadus/items/c2f864c0cde3d17b7efb

## ライブラリなどの準備

In [1]:
#ライブラリ
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation

#GPUの仕様に関する設定
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(
        visible_device_list = "0",
        allow_growth = True,
        per_process_gpu_memory_fraction = 0.1))
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


## データセットの生成

In [2]:
#学習データセット
data = np.random.rand(10000,5) #1000個の5次元ベクトル
labels = (np.sum(data, axis=1) > 2.5) * 1 #ラベル(0,1)
labels = np_utils.to_categorical(labels)  #ラベル(onehot)
#onehotラベルはこんな感じ
#学習データ
print("学習データ")
print(data[:3])
print("正解(onehot)")
print(labels[:3])

学習データ
[[0.215959   0.55337008 0.66038375 0.09465502 0.11583382]
 [0.40788529 0.42522187 0.05672631 0.15324927 0.79814661]
 [0.32200818 0.80420858 0.86132459 0.27578893 0.62284146]]
正解(onehot)
[[1. 0.]
 [1. 0.]
 [0. 1.]]


## ネットワークの構築
kerasでは層を積み上げていくイメージ  
model.add以外にもこんな書き方もできる
```
model = Sequential([
    Dense(20, input_dim=5, activation='relu'),
    Dense(2,activation='softmax')
])
```
Functional APIを使った記法  
複雑なモデルを作るときに使用，最初はこんなのあるんだ見たいな気持ちでOK
```
from keras.layers import Input, Dense, Activation
from keras.models import Model
inputs = Input(shape=(5,))
x = Dense(20)(inputs)
x = Activation('relu')(x)
outputs = Dense(2,activation='softmax')(x)
model = Model(inputs=inputs, outputs = outputs)
```
### 活性化関数
- sigmoid
- relu
- tanh  

### バッチ正規化層
ミニバッチ内での直前の出力を正規化する
- 学習を早く進行できる
- 初期値に依存しない
- 過学習抑制
```
from keras.layers.normalization import BatchNormalization
```

In [3]:
#Sequential:層を積み上げる単純なモデル
model = Sequential()
model.add(Dense(20, input_dim=5))
model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

#ネットワーク構造の出力
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                120       
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


## 学習
### 最適化関数(optimizer)  
- SGD(確率的勾配降下法):  
傾きに比例して移動する  
- Momentam:  
ボールが転がるイメージ  
- AdaGrad:  
学習係数を下げていく(最初は大きく動かし，終盤は微調整)
- Adam:  
MomentumとAdaGradの融合    

例(optimizerを設定する場合
```
from keras import optimizers
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999,epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = adam, 'categorical_crossentropy', metrics=['accuracy'])
```
### 損失関数(loss)
- mean_squared_errer:二乗平均誤差
- mean_absolute_error:絶対平均誤差
- hinge:マイナスのところは0,プラスのところはその値
- categorical_crossentropy:複数クラスの交差エントロピー

In [4]:
#compli:最適化関数,損失関数,評価指標
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
#学習
#fit;入力，出力，バッチサイズ(一度の更新に使用するデータ数), エポック数(学習回数),検証セットの割合(ランダムではないので注意)
model.fit(data, labels, batch_size = 100, epochs=150, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 1s 83us/step - loss: 0.6637 - acc: 0.5581 - val_loss: 0.6269 - val_acc: 0.7230
Epoch 2/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.5895 - acc: 0.7476 - val_loss: 0.5474 - val_acc: 0.8090
Epoch 3/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.5092 - acc: 0.8484 - val_loss: 0.4633 - val_acc: 0.9205
Epoch 4/150
8000/8000 [==============================] - 0s 45us/step - loss: 0.4279 - acc: 0.9160 - val_loss: 0.3863 - val_acc: 0.9485
Epoch 5/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.3586 - acc: 0.9504 - val_loss: 0.3249 - val_acc: 0.9525
Epoch 6/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.3040 - acc: 0.9623 - val_loss: 0.2788 - val_acc: 0.9830
Epoch 7/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.2624 - acc: 0.9764 - val_loss: 0.2415 - val_acc: 0.9835


8000/8000 [==============================] - 0s 44us/step - loss: 0.0467 - acc: 0.9948 - val_loss: 0.0444 - val_acc: 0.9915
Epoch 61/150
8000/8000 [==============================] - 0s 45us/step - loss: 0.0461 - acc: 0.9936 - val_loss: 0.0467 - val_acc: 0.9875
Epoch 62/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0457 - acc: 0.9951 - val_loss: 0.0418 - val_acc: 0.9980
Epoch 63/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0449 - acc: 0.9956 - val_loss: 0.0413 - val_acc: 0.9985
Epoch 64/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0442 - acc: 0.9958 - val_loss: 0.0415 - val_acc: 0.9955
Epoch 65/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0440 - acc: 0.9958 - val_loss: 0.0404 - val_acc: 0.9975
Epoch 66/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0433 - acc: 0.9961 - val_loss: 0.0400 - val_acc: 0.9975
Epoch 67/150
8000/8000 [==============================

Epoch 120/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0294 - acc: 0.9973 - val_loss: 0.0263 - val_acc: 0.9980
Epoch 121/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0291 - acc: 0.9979 - val_loss: 0.0262 - val_acc: 0.9985
Epoch 122/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0292 - acc: 0.9970 - val_loss: 0.0260 - val_acc: 0.9975
Epoch 123/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0288 - acc: 0.9969 - val_loss: 0.0259 - val_acc: 0.9975
Epoch 124/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.0287 - acc: 0.9975 - val_loss: 0.0258 - val_acc: 0.9985
Epoch 125/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.0284 - acc: 0.9964 - val_loss: 0.0262 - val_acc: 0.9960
Epoch 126/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.0288 - acc: 0.9958 - val_loss: 0.0265 - val_acc: 0.9945
Epoch 127/150
8000/8000 [=========

## 未知データの予測

In [5]:
#評価データセット
test_data = np.random.rand(1000,5)
test_label = (np.sum(test_data, axis=1) > 2.5) * 1 

In [6]:
print("入力")
print(test_data[0])
print("正解")
print(test_label[0])
print("予測結果の確率(モデルの出力)")
#出力結果の確率
print(model.predict(test_data[0:1]))
print("確率が最大の次元")
#最も大きい次元を出力とする
print(np.argmax(model.predict(test_data[0:1])))

入力
[0.36840927 0.81994377 0.97752692 0.58652631 0.46108173]
正解
1
予測結果の確率(モデルの出力)
[[1.315631e-12 1.000000e+00]]
確率が最大の次元
1


In [7]:
predict = np.argmax(model.predict(test_data), axis=1)
#predictの中身
print("各データに対する予測")
print(predict[:3])
print("正解と予測の比較")
print((predict==test_label)[:3])
#sumを取るとtrueの数を集計する
print("認識率")
print(sum(predict == test_label) /1000)

各データに対する予測
[1 1 0]
正解と予測の比較
[ True  True  True]
認識率
0.998


## モデルの中身(重みとバイアスを取得)

In [8]:
weight = model.get_weights()
print("hidden weight")
print(weight[0]) #隠れ層重み
print("hidden bias")
print(weight[1]) #隠れ層バイアス
print("output weight")
print(weight[2]) #出力層重み
print("output bias")
print(weight[3]) #出力層バイアス

hidden weight
[[ 0.51173526  1.0018353   0.57670444  0.61433935  0.9116411  -0.8732202
   0.17381035  0.37819323 -0.0645324  -0.03001211  0.820435    0.36244708
  -0.20823146 -0.92751247 -0.22802316 -0.53514665  0.71012837 -0.14226344
  -0.53342736  0.05144089]
 [ 0.33187523  0.9515459   0.74958265  0.75199777  0.5067764  -0.23733686
   0.8387133   0.87421024 -0.19930895 -0.50276744  0.28844628  0.807447
  -0.96072555 -0.02816224  0.03504791  0.2554761   0.7117266  -0.3243068
   0.25391275 -0.3079334 ]
 [ 0.6956371   0.51399183  0.5548462   0.5885558   0.771965    0.09950172
   0.8723312   0.88148767 -0.04893757 -0.52685255  0.42533126  1.066739
  -0.31564638 -0.10009059 -0.290862   -0.6926332   0.3396987  -0.0522249
  -0.2806624  -0.41984835]
 [ 0.6562775   0.54395247  0.77619106  0.20315564  0.27991977  0.12387625
   0.89597297  0.2786703  -1.0023075  -0.37463823  0.96487063  0.9500709
  -0.18731971 -0.21253286 -0.33248508 -0.26711047  0.5575839  -0.41773677
  -0.5028046  -0.6336986 